In [1]:
import sys
sys.path.insert(0, '/cndd/fangming/CEMBA/snmcseq_dev')

from __init__ import *
from snmcseq_utils import cd
from snmcseq_utils import create_logger
# from CEMBA_update_mysql import connect_sql

In [2]:
log = create_logger()
log.info("Hello")

05/20/2020 04:57:10 PM Hello


In [3]:
# 
input_f = '/cndd2/fangming/projects/scf_enhancers/data/bulk/round2/mc/dmr/cgdmr_round2_rms_results_collapsed.tsv'
df = pd.read_table(input_f, index_col=['#chr', 'start', 'end'], dtype={'#chr': object})
print(df.shape)
df.head()

/cndd/fangming/venvs/conda_dobro/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  This is separate from the ipykernel package so we can avoid doing imports until
/cndd/fangming/venvs/conda_dobro/lib/python3.6/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


(1460167, 32)


number_of_dms hypermethylated_samples  \
#chr start   end                                              
chr1 3013472 3013579              3                     NaN   
     3027398 3027514              2                     NaN   
     3033712 3033712              1                     NaN   
     3034471 3034471              1                     NaN   
     3037825 3037825              1                     NaN   

                     hypomethylated_samples  methylation_level_1-1  \
#chr start   end                                                     
chr1 3013472 3013579        5-1,5-3,1-1,9-2               0.561404   
     3027398 3027514                    4-2               0.600000   
     3033712 3033712                6-2,8-3               1.000000   
     3034471 3034471                    6-2               0.705882   
     3037825 3037825                4-2,6-4               1.000000   

                      methylation_level_1-2  methylation_level_10-1  \
#chr start   end                                                      
chr1 3013472 3013579               0.762887                     1.0   
     3027398 3027514               0.678571                     1.0   
     3033712 3033712               0.962963                     NaN   
     3034471 3034471               0.848485                     NaN   
     3037825 3037825               1.000000                     NaN   

                      methylation_level_2-1  methylation_level_2-2  \
#chr start   end                                                     
chr1 3013472 3013579               0.865169               0.955882   
     3027398 3027514               0.730769               0.642857   
     3033712 3033712               0.936170               1.000000   
     3034471 3034471               0.935484               0.923077   
     3037825 3037825               1.000000               1.000000   

                      methylation_level_2-3  methylation_level_3-1  ...  \
#chr start   end                                                    ...   
chr1 3013472 3013579               0.795918               0.833333  ...   
     3027398 3027514               0.714286               0.730769  ...   
     3033712 3033712               1.000000               0.956522  ...   
     3034471 3034471               0.950000               0.920000  ...   
     3037825 3037825               0.964286               1.000000  ...   

                      methylation_level_6-4  methylation_level_6-5  \
#chr start   end                                                     
chr1 3013472 3013579               1.000000               0.904762   
     3027398 3027514               0.625000               0.571429   
     3033712 3033712               0.833333               1.000000   
     3034471 3034471               0.500000               0.625000   
     3037825 3037825               0.846154               1.000000   

                      methylation_level_6-6  methylation_level_7-1  \
#chr start   end                                                     
chr1 3013472 3013579                    NaN               0.973684   
     3027398 3027514                    NaN               0.806452   
     3033712 3033712                    NaN               0.961538   
     3034471 3034471                   0.75                    NaN   
     3037825 3037825                    NaN               1.000000   

                      methylation_level_7-2  methylation_level_8-1  \
#chr start   end                                                     
chr1 3013472 3013579               1.000000                  1.000   
     3027398 3027514               0.833333                  0.875   
     3033712 3033712               1.000000                  1.000   
     3034471 3034471               1.000000                    NaN   
     3037825 3037825               1.000000                    NaN   

                      methylation_level_8-2  methylation_level_8-3  \
#chr start   end               

In [4]:
print(df.shape)

df_hypo = df.loc[(df['number_of_dms']>=3) & (~df['hypomethylated_samples'].isnull()), 'hypomethylated_samples'].apply(lambda x: x.split(','))
df_hyper = df.loc[(df['number_of_dms']>=3) & (~df['hypermethylated_samples'].isnull()), 'hypermethylated_samples'].apply(lambda x: x.split(','))

print(df_hypo.shape)
print(df_hyper.shape)

(1460167, 32)
(592542,)
(519496,)


In [5]:
df_hypo.head()
# df_hypo_cluster = df_hypo.loc[
#         df_hypo.apply(lambda x: ('merged_mCG_{}_{}_{}'.format(cluster_type, i+1, ens) in x))]

#chr  start    end    
chr1  3013472  3013579              [5-1, 5-3, 1-1, 9-2]
      3094063  3094121                   [1-2, 3-1, 5-2]
      3109231  3109390                             [1-2]
      3112234  3112467         [6-2, 6-4, 6-6, 5-3, 6-3]
      3113717  3114035    [2-1, 2-2, 6-2, 2-3, 3-2, 8-1]
Name: hypomethylated_samples, dtype: object

In [10]:
outdir = '/cndd2/fangming/projects/scf_enhancers/enhancer_round2_191211/dmr'
if not os.path.isdir(outdir):
    os.makedirs(outdir)
    logging.info('Created directory {}'.format(outdir))
    
clsts = [i[len('methylation_level_'):] for i in df.filter(regex='^methylation_level_*').columns]
for clst in clsts:
    df_hypo_cluster = df_hypo[df_hypo.apply(lambda x: (clst in x))]
    output = os.path.join(outdir, 'dmr_{}.bed'.format(clst))
    df_hypo_cluster.to_csv(output, sep='\t', header=False, index=True, na_rep='NA')
    logging.info("Saved to {}".format(output))
    logging.info(df_hypo_cluster.shape)

01/23/2020 02:07:37 PM Saved to /cndd2/fangming/projects/scf_enhancers/enhancer_round2_191211/dmr/dmr_1-1.bed
01/23/2020 02:07:37 PM (207413,)
01/23/2020 02:07:40 PM Saved to /cndd2/fangming/projects/scf_enhancers/enhancer_round2_191211/dmr/dmr_1-2.bed
01/23/2020 02:07:40 PM (258769,)
01/23/2020 02:07:41 PM Saved to /cndd2/fangming/projects/scf_enhancers/enhancer_round2_191211/dmr/dmr_10-1.bed
01/23/2020 02:07:41 PM (18082,)
01/23/2020 02:07:44 PM Saved to /cndd2/fangming/projects/scf_enhancers/enhancer_round2_191211/dmr/dmr_2-1.bed
01/23/2020 02:07:44 PM (234123,)
01/23/2020 02:07:46 PM Saved to /cndd2/fangming/projects/scf_enhancers/enhancer_round2_191211/dmr/dmr_2-2.bed
01/23/2020 02:07:46 PM (141155,)
01/23/2020 02:07:48 PM Saved to /cndd2/fangming/projects/scf_enhancers/enhancer_round2_191211/dmr/dmr_2-3.bed
01/23/2020 02:07:48 PM (153768,)
01/23/2020 02:07:51 PM Saved to /cndd2/fangming/projects/scf_enhancers/enhancer_round2_191211/dmr/dmr_3-1.bed
01/23/2020 02:07:51 PM (196704,)

In [7]:
outdir = '/cndd2/fangming/projects/scf_enhancers/enhancer_round2_hyper_200226/dmr'
if not os.path.isdir(outdir):
    os.makedirs(outdir)
    logging.info('Created directory {}'.format(outdir))
    
clsts = [i[len('methylation_level_'):] for i in df.filter(regex='^methylation_level_*').columns]
for clst in clsts:
    df_hyper_cluster = df_hyper[df_hyper.apply(lambda x: (clst in x))]
    output = os.path.join(outdir, 'dmr_{}.bed'.format(clst))
    df_hyper_cluster.to_csv(output, sep='\t', header=False, index=True, na_rep='NA')
    logging.info("Saved to {}".format(output))
    logging.info(df_hyper_cluster.shape)

02/26/2020 09:03:09 PM Saved to /cndd2/fangming/projects/scf_enhancers/enhancer_round2_hyper_200226/dmr/dmr_1-1.bed
02/26/2020 09:03:09 PM (61715,)
02/26/2020 09:03:10 PM Saved to /cndd2/fangming/projects/scf_enhancers/enhancer_round2_hyper_200226/dmr/dmr_1-2.bed
02/26/2020 09:03:10 PM (98223,)
02/26/2020 09:03:11 PM Saved to /cndd2/fangming/projects/scf_enhancers/enhancer_round2_hyper_200226/dmr/dmr_10-1.bed
02/26/2020 09:03:11 PM (20988,)
02/26/2020 09:03:13 PM Saved to /cndd2/fangming/projects/scf_enhancers/enhancer_round2_hyper_200226/dmr/dmr_2-1.bed
02/26/2020 09:03:13 PM (175655,)
02/26/2020 09:03:15 PM Saved to /cndd2/fangming/projects/scf_enhancers/enhancer_round2_hyper_200226/dmr/dmr_2-2.bed
02/26/2020 09:03:15 PM (148417,)
02/26/2020 09:03:16 PM Saved to /cndd2/fangming/projects/scf_enhancers/enhancer_round2_hyper_200226/dmr/dmr_2-3.bed
02/26/2020 09:03:16 PM (74596,)
02/26/2020 09:03:17 PM Saved to /cndd2/fangming/projects/scf_enhancers/enhancer_round2_hyper_200226/dmr/dmr_3

# Not organized 

In [21]:
with cd(os.path.join(PATH_ENSEMBLES, ens, 'dmr')):
    f = 'dmr_allc_merged_mCG_cluster_mCHmCG_lv_npc50_k30_rms_results_collapsed.tsv.DMR.bed'
    df_bed = pd.read_table(f, header=None, names=['chr', 'start', 'end', 'num_dms'])
print(df_bed.shape)
df_bed.head()

(1162175, 4)


,chr,start,end,num_dms
0,chr1,3013472,3013473,1
1,chr1,3023074,3023075,1
2,chr1,3027397,3027398,1
3,chr1,3034471,3034472,1
4,chr1,3044312,3044403,3


In [23]:
df_out = df_bed[df_bed['num_dms']>=20]
with cd(os.path.join(PATH_ENSEMBLES, ens, 'dmr')):
    df_out.to_csv('dmr_allc_merged_mCG_cluster_mCHmCG_lv_npc50_k30_rms_results_collapsed_20dms.tsv.DMR.bed', 
                 sep='\t', na_rep='NA', header=False, index=False)